In [1]:
pip install spacy


Note: you may need to restart the kernel to use updated packages.


In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.en import English
import numpy as np

In [2]:

nlp = English()
nlp.add_pipe(nlp.create_pipe('sentencizer'))



In [4]:
from pathlib import Path
text_corpus = Path('d.txt').read_text()
text_corpus = text_corpus.replace('\n', '')

In [5]:
doc = nlp(text_corpus.replace("\n", ""))
sentences = [sent.string.strip() for sent in doc.sents]

In [6]:
print("Senetence are: \n", sentences) 

Senetence are: 
 ["Google celebrated British illustrator and artist Sir John Tenniel's 200th birth anniversary with a doodle on February 28.", "An acclaimed Victorian painter, Tenniel is celebrated for his illustrations for Lewis Carroll's Alice's Adventures in Wonderland and Through the Looking-Glass.", 'Tenniel was born in Bayswater, West London in 1820.', 'At the age of 20, Tenniel received a major eye injury and eventually, lost sight in his right eye.', 'From a very early age, Tenniel was appreciated as a humorist and soon after, also cultured his talent for scholarly caricature.', "His first illustration was for Samuel Carter Hall's The Book of British Ballads in 1842.", 'Eight years later, he joined the historic weekly magazine Punch as a political cartoonist.', "Lewis Carroll noticed Tenniel's distinct style of work and in 1864, approached the artist to illustrate his book, Alice's Adventures in Wonderland.", 'This association marked Carroll and Tenniel\'s creative partnership 

In [7]:
# Let's create an organizer which will store the sentence ordering to later reorganize the 
# scored sentences in their correct order
sentence_organizer = {k:v for v,k in enumerate(sentences)}

In [8]:
print("Our sentence organizer: \n", sentence_organizer)

Our sentence organizer: 
 {"Google celebrated British illustrator and artist Sir John Tenniel's 200th birth anniversary with a doodle on February 28.": 0, "An acclaimed Victorian painter, Tenniel is celebrated for his illustrations for Lewis Carroll's Alice's Adventures in Wonderland and Through the Looking-Glass.": 1, 'Tenniel was born in Bayswater, West London in 1820.': 2, 'At the age of 20, Tenniel received a major eye injury and eventually, lost sight in his right eye.': 3, 'From a very early age, Tenniel was appreciated as a humorist and soon after, also cultured his talent for scholarly caricature.': 4, "His first illustration was for Samuel Carter Hall's The Book of British Ballads in 1842.": 5, 'Eight years later, he joined the historic weekly magazine Punch as a political cartoonist.': 6, "Lewis Carroll noticed Tenniel's distinct style of work and in 1864, approached the artist to illustrate his book, Alice's Adventures in Wonderland.": 7, 'This association marked Carroll and

In [9]:
# Let's now create a tf-idf (Term frequnecy Inverse Document Frequency) model
tf_idf_vectorizer = TfidfVectorizer(min_df=2,  max_features=None, 
                                    strip_accents='unicode', 
                                    analyzer='word',
                                    token_pattern=r'\w{1,}',
                                    ngram_range=(1, 3), 
                                    use_idf=1,smooth_idf=1,
                                    sublinear_tf=1,
                                    stop_words = 'english')

In [10]:
# Passing our sentences treating each as one document to TF-IDF vectorizer
tf_idf_vectorizer.fit(sentences)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=2, ngram_range=(1, 3), norm='l2', preprocessor=None,
                smooth_idf=1, stop_words='english', strip_accents='unicode',
                sublinear_tf=1, token_pattern='\\w{1,}', tokenizer=None,
                use_idf=1, vocabulary=None)

In [11]:
# Transforming our sentences to TF-IDF vectors
sentence_vectors = tf_idf_vectorizer.transform(sentences)

In [13]:
# Getting sentence scores for each sentences
sentence_scores = np.array(sentence_vectors.sum(axis=1)).ravel()

# Sanity checkup
print(len(sentences) == len(sentence_scores))# Getting top-n sentences
N = 3
top_n_sentences = [sentences[ind] for ind in np.argsort(sentence_scores, axis=0)[::-1][:N]]

True


In [14]:

# Getting top-n sentences
N = 3
top_n_sentences = [sentences[ind] for ind in np.argsort(sentence_scores, axis=0)[::-1][:N]]

In [15]:
# Let's now do the sentence ordering using our prebaked sentence_organizer
# Let's map the scored sentences with their indexes
mapped_top_n_sentences = [(sentence,sentence_organizer[sentence]) for sentence in top_n_sentences]
print("Our top_n_sentence with their index: \n")
for element in mapped_top_n_sentences:
    print(element)

# Ordering our top-n sentences in their original ordering
mapped_top_n_sentences = sorted(mapped_top_n_sentences, key = lambda x: x[1])
ordered_scored_sentences = [element[0] for element in mapped_top_n_sentences]

# Our final summary
summary = " ".join(ordered_scored_sentences)

Our top_n_sentence with their index: 

("An acclaimed Victorian painter, Tenniel is celebrated for his illustrations for Lewis Carroll's Alice's Adventures in Wonderland and Through the Looking-Glass.", 1)
("Lewis Carroll noticed Tenniel's distinct style of work and in 1864, approached the artist to illustrate his book, Alice's Adventures in Wonderland.", 7)
("Google celebrated British illustrator and artist Sir John Tenniel's 200th birth anniversary with a doodle on February 28.", 0)


In [16]:
print("Summary: \n", summary)

Summary: 
 Google celebrated British illustrator and artist Sir John Tenniel's 200th birth anniversary with a doodle on February 28. An acclaimed Victorian painter, Tenniel is celebrated for his illustrations for Lewis Carroll's Alice's Adventures in Wonderland and Through the Looking-Glass. Lewis Carroll noticed Tenniel's distinct style of work and in 1864, approached the artist to illustrate his book, Alice's Adventures in Wonderland.
